<a href="https://colab.research.google.com/github/MaximArtemev/hse_gans/blob/master/seminars/seminar-5_6/5_6.vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import comet_ml in the top of your file
from comet_ml import Experiment
    
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="lODeHEtCf7XLaV6DJrOfugNcA",
                        project_name="hse-gans-vae-1", workspace="holybayes")

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.utils import make_grid


seed=1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/holybayes/hse-gans-vae-1/81406501a09d4450b908ce95a074fb8b



# Dataset
MNIST dataset will be used

In [2]:
batch_size=128
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
train_loader = ??? #
test_loader = ??? # Define dataloaders for mnist here

## AE model

In [3]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.encoder, self.decoder = ???# your code here

    def encode(self, x): 
        x = x.view(-1,28*28)
        ???# your code here
    def decode(self, z): 
        x = x.view(-1,28*28)
        ???# your code here
    def forward(self, x): ???# your code here
    def sample(self, size):  ???# your code here
    
    @property
    def device(self): return next(self.parameters()).device


model = AE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## Train model

In [5]:
def recon_loss(recon_x, x):
    ??? # Loss for AE


log_interval=10
epochs=10

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        ??? # implement gradient step here
        if batch_idx % log_interval == 0:
            experiment.log_metric('ae/train/loss', loss.item() / len(data), step=batch_idx, epoch=epoch)
            
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    ??? # define 
                
    test_loss /= len(test_loader.dataset)
    experiment.log_metric('ae/test/loss', test_loss, epoch=epoch)

for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample_size=64
        sample = model.sample(sample_size).cpu()
        log_img = make_grid(sample.view(-1,1,28,28)).permute((1,2,0))
        experiment.log_image(log_img,name='ae/sample',step=epoch)

====> Epoch: 1 Average loss: 28.2380
====> Epoch: 2 Average loss: 11.4331
====> Epoch: 3 Average loss: 9.5636
====> Epoch: 4 Average loss: 8.7124
====> Epoch: 5 Average loss: 8.1656
====> Epoch: 6 Average loss: 7.7557
====> Epoch: 7 Average loss: 7.4489
====> Epoch: 8 Average loss: 7.1876
====> Epoch: 9 Average loss: 6.9836
====> Epoch: 10 Average loss: 6.7976


## VAE model

In [6]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.fc1 = nn.Linear(784, 400)
        ??? # mu head
        ??? # sigma head
        
        # Decoder
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        mu, logsigma = ??? # your code here
        return mu, logsigma

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    def sample(self, size):
        ??? # sample from N(0,1)
    
    @property
    def device(self): return next(self.parameters()).device


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## Train VAE
Реализуйте вариационный автокодировщик (VAE)

**Подсказка 1**:

$$L = \int_zq(z)log(p(x|\Theta,z))dz - KL(q(z)||p(z))$$

**Подсказка 2**:

$$p(z)=\mathcal{N}(0,1)$$

$$q(z)=\mathcal{N}(\mu_1,\Sigma_1)$$

![](img/KL.png)

(k-размерность)

In [7]:
def loss_function(recon_x, x, mu, logsigma):
    MSE = ??? # first term of L
    KLD = ??? # second term
    return MSE, KLD


log_interval=10
epochs=10

def train(epoch):
    model.train()
    train_mse, train_kld, train_loss = 0, 0, 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        ??? # zero gradients from previous batch
        recon_batch, mu, logvar = ??? # predict for batch using VAE
        mse_loss, kld_loss = ??? # define loss
        loss = mse_loss + kld_loss # it differs from formula above because we are MINIMIZING loss
        ??? # perform gradient step
        train_loss += loss.item();
        train_mse += mse_loss.item(); train_kld += kld_loss.item()
        if batch_idx % log_interval == 0:
            experiment.log_metrics({'vae/train/mse': mse_loss.item()/len(data),
                                    'vae/train/kld': kld_loss.item()/len(data),
                                    'vae/train/loss': loss.item()/len(data)
                                   }, step=batch_idx, epoch=epoch)

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    test_loss,kld_loss,mse_loss = ??? # your code here
    experiment.log_metrics({'vae/test/loss': test_loss, 'vae/test/kld': kld_loss, 'vae/test/mse': mse_loss}, epoch=epoch)
    

for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = model.sample(64).cpu()
        log_img = make_grid(sample.view(-1,1,28,28)).permute((1,2,0))
        experiment.log_image(log_img,name='vae/sample',step=epoch)

====> Epoch: 1 Average loss: 45.4823
====> Epoch: 2 Average loss: 35.0653
====> Epoch: 3 Average loss: 33.2043
====> Epoch: 4 Average loss: 32.3159
====> Epoch: 5 Average loss: 31.7516
====> Epoch: 6 Average loss: 31.4142
====> Epoch: 7 Average loss: 31.1500
====> Epoch: 8 Average loss: 30.9440
====> Epoch: 9 Average loss: 30.7579
====> Epoch: 10 Average loss: 30.6381


# Задания
* сравните качество полученных картинок от AE и VAE
* что можно сказать по полученным результатам?
* чем можно объяснить разницу в качестве?
* обучите на cifar и celeba

# Links

[simple explanation (towardsds)](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)

[ELBO inference](https://chrisorm.github.io/VI-ELBO.html)